In [0]:
df = spark.read.parquet("dbfs:/mnt/Anac/Silver/anac_silver.parquet/")


In [0]:
# 1°: Selecionar somente colunas que o cliente/setor pediu 

colunas = ['Aerodromo_de_Destino', 'Aerodromo_de_Origem','Classificacao_da_Ocorrência', 'Danos_a_Aeronave', 'Data_da_Ocorrencia','Municipio','UF','Regiao','Tipo_de_Aerodromo','Tipo_de_Ocorrencia','Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes' ]

df = df.select(colunas)


In [0]:
# 2°: Criar uma nova coluna que faz a soma de todas as lesões 
colunas_lesoes = ['Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes']

df = df.withColumn("Total_Lesoes", sum(df[col] for col in colunas_lesoes))


In [0]:
# 3°: Renomear colunas para ficar mais intuitivas para o usuário final 
df = df.withColumnRenamed('Aerodromo_de_Destino','Destino')\
    .withColumnRenamed('Aerodromo_de_Origem','Origem')\
        .withColumnRenamed('Classificacao_da_Ocorrência','Classificacao')\
            .withColumnRenamed('Danos_a_Aeronave','Danos')\
                .withColumnRenamed('Data_da_Ocorrencia','Data')\
                    .withColumnRenamed('UF','Estado')




In [0]:
# 4°: Excluir dados que estados tenham a classificação [Indeterminado, Sem Registro, Exterior] 
classificacao_excluir = ["Indeterminado","Sem Registro","Exterior"]
df = df.filter(~df['Estado'].isin(classificacao_excluir))


In [0]:
# 5°: Inserir coluna com nome de atualização para usuário ver quando os dados foram atualizados 
from pyspark.sql.functions import current_timestamp, date_format,from_utc_timestamp

df = df.withColumn("Atualizacao",
                   date_format(from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"),"yyyy-MM-dd HH:mm:ss"))


In [0]:
# 6°: Salvar na camada Gold particionada por UF > 'Estado'
df.write\
    .mode("overwrite")\
        .format("parquet")\
            .partitionBy("Estado")\
                .save("dbfs:/mnt/Anac/Gold/anac_gold_particionado")
